# Topics in Quantitative Finance - Homework 3

Assigned: Friday, July 28, 2023.
Due: **Monday, July 31, 2023** by 1PM. 

Late homework **will not be accepted**.

$$
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
%\newcommand{\Pr}{\mathrm{Pr} }
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\RR}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\sgn}{\mathrm{sgn} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
%\newcommand{\HS}{\mathrm{HS} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
%\newcommand{\LL}{\mathrm{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\p}{\partial}
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\BS}{\text{BS}}
$$

<font color = "red">Homework is to be done by each student individually.  To receive full credit, you must email a completed copy of this Jupyter notebook to TAs 郭鵬 at [g.p@pku.edu.cn](mailto:g.p@pku.edu.cn) and 姜佳文 at [1164063524@qq.com](mailto:1164063524@qq.com) by the due date and time.  All codes must run correctly and solutions must be written up neatly in Markdown/LaTeX format. If you encounter problems with Jupyter notebook, please contact TA [郭鵬](mailto:g.p@pku.edu.cn).

## Name: <font color=blue>彭扬慧</font>

## Delta and delta-gamma hedging

### 1. (25 points)

A portfolio $\Pi$ consists of 
- a long position in 200 shares of a nondividend paying stock with spot price $\$50$,
- a short position in 2000 calls struck at $\$45$ and expiring in 2 months on the stock, assumed lognormally distributed with $40\%$ volatility, 
- a short position in 1000 puts struck at $\$55$ and expiring in 3 months on the same stock
- $\$20,000$ in money account with annual interest rate $5\%$ continuously compounding.

Answer the following questions.

* (a) What is the value of the portfolio $\Pi$?
* (b) How do you adjust the holdings of stock shares and cash amounts in the portfolio $\Pi$ in order to make it delta neutral without changing the postion in calls?
* (c) How do you adjust the portfolio $\Pi$ in order to make it delta-gamma neutral by adding position in calls struck at $\$60$, expiring in 3 months? Position in calls cannot be altered.
* (d) A month later the stock price goes down to $\$48$. Determine the value of the delta-neuralized portfolio in (b). 
* (e) How do you rebalance the portfolio in (d) so it remains delta neutral?    


You may consider using the code provided in the cell below for the calculation of deltas and gammas of call and put. 

In [1]:
# as always, import required modules and functions
import numpy as np
from numpy import sqrt, log, exp
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy.stats import norm
import seaborn as sns

In [2]:
# Black-Scholes formulas
# call
def bs_call(s, K, sigma, t, r=0, d=0):
    d1 = (log(s/K) + (r - d)*t)/(sigma*sqrt(t)) + sigma*sqrt(t)/2
    d2 = d1 - sigma*sqrt(t)
    
    c = s*exp(-d*t)*norm.cdf(d1) - K*exp(-r*t)*norm.cdf(d2)
    delta = exp(-d*t)*norm.cdf(d1)
    gamma = norm.pdf(d1)/s/sigma/sqrt(t)
    
    return {'c': c, 'delta': delta, 'gamma': gamma}

#put
def bs_put(s, K, sigma, t, r=0, d=0):
    d1 = (log(s/K) + (r - d)*t)/(sigma*sqrt(t)) + sigma*sqrt(t)/2
    d2 = d1 - sigma*sqrt(t)
    
    p = K*exp(-r*t)*norm.cdf(-d2) - s*exp(-d*t)*norm.cdf(-d1)
    delta = -exp(-d*t)*norm.cdf(-d1)
    gamma = norm.pdf(d1)/s/sigma/sqrt(t)
    
    return {'p': p, 'delta': delta, 'gamma': gamma}

In [3]:
# parameters
s, K1, K2, r, sig, t1, t2 = 50, 45, 55, 0.05, 0.4, 2/12, 3/12

# positions in underlying, call, put, cash
x, y1, y2, z = 200, -2_000, -1_000, 20_000

### <font color=blue> Solution 1. </font>

In [4]:
#(a) Calculate the value of the portfolio
call1 = bs_call(s, K1, sig, t1, r)
put1 = bs_put(s, K2, sig, t2, r)

port_value = x * s + y1 * call1['c'] + y2 * put1['p'] + z
print(f"The value of the portfolio is : {port_value}")

The value of the portfolio is : 10376.082376326991


In [5]:
#(b)
delta_x = -(x + y1 * call1['delta'] + y2 * put1['delta'])
delta_z = (- delta_x) * s

print(f"Adjust the stock position by {delta_x:.2f} shares.")
print(f"Adjust the cash amount by ${delta_z:.2f}.")

Adjust the stock position by 740.10 shares.
Adjust the cash amount by $-37005.06.


In [6]:
#(c)
K3 = 60
t3 = 3/12
# Recalculate call
call2 = bs_call(s, K3, sig, t3, r)
# Calculate Gamma、stock share、cash amount
G = y1 * call1['gamma'] + y2 * put1['gamma']
y3 = - G/call2['gamma']
x1 = - (y1 * call1['delta'] + y2 * put1['delta'] + y3 * call2['delta'])
z1 = x * s + z - x1 * s - y3 * call2['c']

print(f"Adjust the stock position by {x1:.2f} shares.")
print(f"Adjust the cash amount by ${z1:.2f}.")

Adjust the stock position by 110.46 shares.
Adjust the cash amount by $20105.54.


In [7]:
#(d)
s2 = 48
t3 = t1 - 1/12
t4 = t2 - 1/12

z2 = (z - delta_z) * np.exp(r/12)
call3 = bs_call(s2, K1, sig, t3, r)
put2 = bs_put(s2, K2, sig, t4, r)
port_value1 = (x1 - delta_x) * s2 + y1 * call3['c'] + y2 * put2['p'] + z2

print(f"The value of the new portfolio is : {port_value1}")

The value of the new portfolio is : 11249.42565254341


In [8]:
#(e)
x2 = -(y1 * call3['delta'] + y2 * put2['delta'])
z3 = (x1 - delta_x) * s2 + z2 - x2 * s2
print(f"Adjust the stock position by {x2:.2f} shares.")
print(f"Adjust the cash amount by ${z3:.2f}.")

Adjust the stock position by 727.74 shares.
Adjust the cash amount by $-7911.11.


## The Bachelier model

### 2. (10 points)

Recall that the Bachelier model, assuming zero risk free and dividend rates, reads

$$
S_t = S_0 + \sigma W_t
$$

where $W_t$ is a Brownian motion and $\sigma$ is a constant. 

- (a) Show that a call option struck at $K$, expiring at $T$ has the price $c$ at time zero given by 

    \begin{eqnarray*}
    c(K, T) &=& \Eof{(S_T - K)^+}\\
    &=& \frac{\sigma\sqrt T}{\sqrt{2\pi}} \, e^{-\frac{(S_0 - K)^2}{2\sigma^2 T}} + (S_0 - K) \, N\left(\frac{S_0 - K}{\sigma\sqrt T}\right).
    \end{eqnarray*}

- (b) Verify the Dupire's formula, in this case 

    $$
    \frac{\sigma^2}{K^2} = \frac{c_T}{\frac12 K^2 c_{KK}}
    $$

    using the call price function $c$ obtained in (a). Note that subindices refer to corresponding partial derivatives. 

### <font color=blue> Solution 2. </font>

(a)
  
  $E(S_T - K) = \int _0^\infty \frac{x}{\sqrt{2\pi} \sigma \sqrt T} 
  e^ {-\frac{(x + K - S_0)^2}{2\sigma^2 T}} dx$
  
  $= \frac{1}{\sqrt{2\pi}} \int_{\frac{K - S_0}{\sigma \sqrt T}}^\infty
  (u + \frac{S_0 - K}{\sigma \sqrt T})e^{-u^2}  d(\sigma \sqrt Tu)$

  $= \frac{\sigma \sqrt T}{\sqrt{2\pi}} \int_{\frac{K - S_0}{\sigma \sqrt T}}^\infty
  ue^{-u^2}du + \frac{S_0 - K}{\sqrt{2\pi}} \int_{-\infty}^\frac{S_0 - K}{\sigma \sqrt T} e^{\frac{-u^2}{2}}du$

  $= \frac{\sigma \sqrt T}{\sqrt{2\pi}} e^{-\frac{u^2}{2}}|_{\frac{K - S_0}{\sigma \sqrt T}}^\infty + (S_0 - K) \int_{-\infty}^\frac{S_0 - K}{\sigma \sqrt T} \frac{1}{\sqrt{2\pi}}e^{-\frac{u^2}{2}}du$

  $= \frac{\sigma\sqrt T}{\sqrt{2\pi}} \, e^{-\frac{(S_0 - K)^2}{2\sigma^2 T}} + (S_0 - K) \, \Phi(\frac{S_0 - K}{\sigma\sqrt T})$


(b)  
$ c_T = \frac{\sigma}{2\sqrt{2\pi T}} e^{-\frac{(S_0-K)^2}{2\sigma^2T}} $  

$ c_{K} = -N(\frac{S_0 - K}{\sigma\sqrt T})$   

$ c_{KK} = \frac{1}{\sqrt{2\pi T}\sigma} e^{-\frac{(S_0-K)^2}{2\sigma^2T}} $  

so $
    \frac{\sigma^2}{K^2} = \frac{c_T}{\frac12 K^2 c_{KK}}
    $